In [2]:
from datasets import load_dataset
import pandas as pd
import torch
import os
from tqdm import tqdm
import ast

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
# load all guide books und travel guides clues
df_all_clues = pd.read_csv("all_clues.csv")
df_all_clues

,CLUE_ID,text,ISO2,clue_type,ISO3
0,0,"In Street View, if the sun is clearly in the N...",[],0,[]
1,1,If the tip of a shadow is pointing south then ...,[],0,[]
2,2,The direction that the shadow’s tip points gen...,[],0,[]
3,3,If the sun is in the Northern Hemisphere then ...,[],0,[]
4,4,On some occasions the sun will appear overhead...,[],0,[]
...,...,...,...,...,...
4115,4115,Subtropical with rapid changes in temperature ...,['PY'],1,['PRY']
4116,4116,The weather in Peru varies according to area –...,['PE'],1,['PER']
4117,4117,"Tropical and humid, cooled by the northeast tr...",['SR'],1,['SUR']
4118,4118,Uruguay has an exceptionally fine temperate cl...,['UY'],1,['URY']


In [5]:
# load one-hot encodings for ISO2 codes
df_one_hot_enc = pd.read_csv("encodings.csv")
df_one_hot_enc

,ISO2,encoding
0,AD,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,AE,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,AF,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,AG,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,AI,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
216,XK,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
217,YE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
218,ZA,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
219,ZM,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
# convert the vectors in 'encoding' column to lists
df_one_hot_enc['encoding'] = df_one_hot_enc['encoding'].apply(ast.literal_eval)
df_one_hot_enc

,ISO2,encoding
0,AD,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,AE,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,AF,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,AG,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,AI,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
216,XK,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
217,YE,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
218,ZA,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
219,ZM,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
# create dic as lookup table
one_hot_enc_dict = df_one_hot_enc.set_index('ISO2')['encoding'].to_dict()
one_hot_enc_dict

In [8]:
# maps ISO2 codes to their respective encodings as lists of lists
def map_iso2_to_encoding(iso2_list: list):
    return [one_hot_enc_dict.get(iso2, []) for iso2 in iso2_list]

In [9]:
# add new column 'country_one_hot_enc' with one_hot encodings to all_clues_df
df_all_clues['country_one_hot_enc'] = df_all_clues['ISO2'].apply(eval).apply(map_iso2_to_encoding)
df_all_clues[:10]

,CLUE_ID,text,ISO2,clue_type,ISO3,country_one_hot_enc
0,0,"In Street View, if the sun is clearly in the N...",[],0,[],[]
1,1,If the tip of a shadow is pointing south then ...,[],0,[],[]
2,2,The direction that the shadow’s tip points gen...,[],0,[],[]
3,3,If the sun is in the Northern Hemisphere then ...,[],0,[],[]
4,4,On some occasions the sun will appear overhead...,[],0,[],[]
5,5,If a satellite dish points south then you are ...,[],0,[],[]
6,6,On occasions I have played a timed round and s...,['BR'],0,['BRA'],"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
7,7,This indicates that we are in the Northern Hem...,[],0,[],[]
8,8,"As a general rule, countries of British origin...","['CA', 'GB', 'IS', 'PH']",0,"['CAN', 'GBR', 'ISL', 'PHL']","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
9,9,"For some reason only the sensible USA, UK and ...","['GB', 'LR', 'MM', 'US']",0,"['GBR', 'LBR', 'MMR', 'USA']","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [ ]:
# load sentence transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2').cuda()

In [15]:
from datasets import Dataset

# creates new dataset with encoded clues from the all_clues_df
def encode_dataset(df, model):
    encodings = []
    # iterate over each row in the df
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Encoding rows"):
        # extract values from the current row
        clue_id = row[0]
        sentences = row[1]
        iso_2 = row[2]
        clue_type = row[3]
        iso_3 = row[4]
        one_hot_enc = row[5]

        # encode text clues using the model 
        sentence_embeddings = model.encode(sentences)

        # append the encodings and extracted values to the list
        encodings.append({"clue_id": clue_id, "text": sentences, "iso2": iso_2, "clue_type": clue_type, "iso3": iso_3, "country_one_hot_enc": one_hot_enc, "encoding": sentence_embeddings} )

     # convert the list to a HuggingFace Dataset
    return Dataset.from_list(encodings)

In [ ]:

# creates new HuggingFace Dataset with of df_all_clues with encoded clues using sentence transformers 
dataset = encode_dataset(df_all_clues, model)


In [ ]:
# push created HuggingFace Dataset to HuggingFace
dataset.push_to_hub("gips-mai/all_clues_enc")